In [1]:
from ch2.data import *

s = session('-v5')
#ff_data = statistics(s, FITNESS, FATIGUE)
hr10_data = statistics(s, HR_IMPULSE_10, with_sources=True)
hr10_data.describe()

    INFO: Using database at /home/andrew/.ch2/database.sqlr


,HR Impulse / 10s,Src HR Impulse / 10s
count,338770.000000,338770.000000
mean,0.055525,158424.035747
std,0.090322,304.140518
min,0.000000,157895.000000
25%,0.000002,158185.000000
50%,0.017111,158414.000000
75%,0.070150,158692.000000
max,1.241608,159092.000000


In [2]:
from ch2.squeal import *

segment_journals = s.query(SegmentJournal).join(Segment).filter(Segment.name == 'San Cristobal').all()
print(segment_journals)
st_data = statistics(s, SEGMENT_TIME, sources=segment_journals)
st_data.describe()
print(st_data)

[<ch2.squeal.tables.segment.SegmentJournal object at 0x7f1977639908>, <ch2.squeal.tables.segment.SegmentJournal object at 0x7f1977639be0>, <ch2.squeal.tables.segment.SegmentJournal object at 0x7f1977639c50>, <ch2.squeal.tables.segment.SegmentJournal object at 0x7f1977639cc0>, <ch2.squeal.tables.segment.SegmentJournal object at 0x7f1977639d30>, <ch2.squeal.tables.segment.SegmentJournal object at 0x7f1977639da0>, <ch2.squeal.tables.segment.SegmentJournal object at 0x7f1977639e10>, <ch2.squeal.tables.segment.SegmentJournal object at 0x7f1977639eb8>, <ch2.squeal.tables.segment.SegmentJournal object at 0x7f1977639f60>, <ch2.squeal.tables.segment.SegmentJournal object at 0x7f197763d048>, <ch2.squeal.tables.segment.SegmentJournal object at 0x7f197763d0f0>, <ch2.squeal.tables.segment.SegmentJournal object at 0x7f197763d198>, <ch2.squeal.tables.segment.SegmentJournal object at 0x7f197763d240>, <ch2.squeal.tables.segment.SegmentJournal object at 0x7f197763d2e8>, <ch2.squeal.tables.segment.Segmen

In [12]:
from math import log10
import pandas as pd
import datetime as dt
from collections import namedtuple
from ch2.data.impulse import *

# scale of 10 because 10 secs between impulses
model = DecayModel(1e-10, 0, log10(10), log10(42 * 24), HR_IMPULSE_10, FITNESS)
hr3600_data = pre_calc(hr10_data, model, target=st_data)
print(hr3600_data.describe())
print(hr3600_data)

                           Segment Time
Index                                  
2016-07-27 12:00:00+00:00   1280.092025
2016-08-25 16:00:00+00:00   1080.567709
2016-09-07 11:00:00+00:00   1124.853833
2016-09-09 11:00:00+00:00   1137.381113
2016-11-29 16:00:00+00:00   1096.811452
2016-12-27 11:00:00+00:00   1147.483675
2017-01-18 11:00:00+00:00   1057.599308
2017-02-09 11:00:00+00:00    971.906796
2017-05-05 10:00:00+00:00   1068.400494
2017-07-12 12:00:00+00:00   1042.145533
2017-07-14 14:00:00+00:00   1011.989244
2017-08-08 12:00:00+00:00    978.233589
2017-08-10 14:00:00+00:00    944.136378
2017-08-12 12:00:00+00:00   1000.695077
2017-10-04 11:00:00+00:00    917.020896
2018-01-12 10:00:00+00:00    933.769402
2018-01-26 11:00:00+00:00    925.666688
2019-05-14 14:00:00+00:00   1310.992192
2019-05-28 16:00:00+00:00   1278.995325
       Fitness  Impulse / 3600s
count  25396.0     25396.000000
mean       0.0         0.740681
std        0.0         4.745326
min        0.0         0.000000


In [4]:
hr3600_data = calc(hr3600_data, model)
print(hr3600_data)

   DEBUG: Exponential decay: 0.9990084284402332; alpha: 0.0009915715597668129; period 1008.0000000000005 (intervals)


                               Fitness  Impulse / 3600s
2016-07-04 14:00:00+00:00     0.000001     1.000000e-10
2016-07-04 15:00:00+00:00   760.962035     7.609620e+01
2016-07-04 16:00:00+00:00  1146.793225     3.865857e+01
2016-07-04 17:00:00+00:00  1145.656098     0.000000e+00
2016-07-04 18:00:00+00:00  1144.520098     0.000000e+00
2016-07-04 19:00:00+00:00  1143.385224     0.000000e+00
2016-07-04 20:00:00+00:00  1142.251476     0.000000e+00
2016-07-04 21:00:00+00:00  1141.118852     0.000000e+00
2016-07-04 22:00:00+00:00  1139.987351     0.000000e+00
2016-07-04 23:00:00+00:00  1138.856972     0.000000e+00
2016-07-05 00:00:00+00:00  1137.727713     0.000000e+00
2016-07-05 01:00:00+00:00  1136.599575     0.000000e+00
2016-07-05 02:00:00+00:00  1135.472555     0.000000e+00
2016-07-05 03:00:00+00:00  1134.346653     0.000000e+00
2016-07-05 04:00:00+00:00  1133.221867     0.000000e+00
2016-07-05 05:00:00+00:00  1132.098196     0.000000e+00
2016-07-05 06:00:00+00:00  1130.975640     0.000

In [5]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.palettes import Dark2
from bokeh.models import LinearAxis, Range1d

output_notebook()
f = figure(plot_width=500, plot_height=450, x_axis_type='datetime')
#f.extra_y_ranges = {'Speed': Range1d(start=-0.000067, end=0.0014)}  # start by hand to align zero
#f.add_layout(LinearAxis(y_range_name='Speed', axis_label='Speed (arbitrary units)'), 'right')
pt = Dark2[3]
#f.line(x=ff_data.index, y=ff_data[FITNESS], color=pt[0], legend=FITNESS)
f.line(x=hr3600_data.index, y=hr3600_data[FITNESS], color=pt[1], legend=f'new {FITNESS}')
#f.circle(x=st_data.index, y=1.0/st_data[SEGMENT_TIME], fill_color=pt[2], size=10, legend='Speed', y_range_name='Speed')
f.legend.location = 'top_left'
show(f)

Loading BokehJS ...

In [9]:
from ch2.data.lib import fit

st_data = st_data.set_index(st_data.index.round('1H'))
data = pre_calc(hr10_data, model, target=st_data)
data = data.join(st_data, how='outer')
data[SPEED] = 1/data[SEGMENT_TIME]

def evaluate(data, model):
    data = calc(data, model)
    data[FITNESS] += model.zero
    return data

data = evaluate(data, model._replace(log10_scale=0))
model = model._replace(log10_scale=log10((data[SPEED] / data[FITNESS]).dropna().mean()), zero=1e-5)
print(model)
result = fit(SPEED, FITNESS, data, model, evaluate, 'log10_scale', 'log10_period', 'zero', tol=1e-10)
print(result)

   DEBUG: Exponential decay: 0.9990084284402332; alpha: 0.0009915715597668129; period 1008.0000000000005 (intervals)
   DEBUG: Exponential decay: 0.9990084284402332; alpha: 0.0009915715597668129; period 1008.0000000000005 (intervals)
   DEBUG: Exponential decay: 0.9990084284402332; alpha: 0.0009915715597668129; period 1008.0000000000005 (intervals)
   DEBUG: Exponential decay: 0.9992981989679436; alpha: 0.000701801032056415; period 1424.4052270722368 (intervals)
   DEBUG: Exponential decay: 0.9990084284402332; alpha: 0.0009915715597668129; period 1008.0000000000005 (intervals)
   DEBUG: Exponential decay: 0.9985990968412769; alpha: 0.0014009031587230991; period 713.3250992685894 (intervals)
   DEBUG: Exponential decay: 0.9980209561256285; alpha: 0.0019790438743715333; period 504.7943425064902 (intervals)
   DEBUG: Exponential decay: 0.9984280866237656; alpha: 0.0015719133762344129; period 635.6672399049288 (intervals)
   DEBUG: Exponential decay: 0.9978630573848596; alpha: 0.0021369426

DecayModel(start=1e-10, zero=1e-05, log10_scale=-5.975618542584628, log10_period=3.0034605321095067, input='HR Impulse / 10s', output='Fitness')


   DEBUG: Exponential decay: 0.9895031544310603; alpha: 0.010496845568939706; period 94.76583609696111 (intervals)
   DEBUG: Exponential decay: 0.9878596668321912; alpha: 0.012140333167808826; period 81.86904446825298 (intervals)
   DEBUG: Exponential decay: 0.9873815116530723; alpha: 0.012618488346927692; period 78.74773707128954 (intervals)
   DEBUG: Exponential decay: 0.9884673812935204; alpha: 0.011532618706479592; period 86.20960842707466 (intervals)
   DEBUG: Exponential decay: 0.9890921606652042; alpha: 0.010907839334795777; period 91.17627059536858 (intervals)
   DEBUG: Exponential decay: 0.9880190810625091; alpha: 0.01198091893749087; period 82.96504730927761 (intervals)
   DEBUG: Exponential decay: 0.987376088409347; alpha: 0.012623911590653014; period 78.71369126014272 (intervals)
   DEBUG: Exponential decay: 0.9884727782119795; alpha: 0.011527221788020459; period 86.25020581961931 (intervals)
   DEBUG: Exponential decay: 0.9887661614796585; alpha: 0.011233838520341544; peri

   DEBUG: Exponential decay: 0.988269638232882; alpha: 0.011730361767117947; period 84.74787941854046 (intervals)
   DEBUG: Exponential decay: 0.9882702576633899; alpha: 0.01172974233661006; period 84.7523813383167 (intervals)
   DEBUG: Exponential decay: 0.9882705586890025; alpha: 0.011729441310997513; period 84.75456931499447 (intervals)
   DEBUG: Exponential decay: 0.9882709233804745; alpha: 0.011729076619525491; period 84.7572201915001 (intervals)
   DEBUG: Exponential decay: 0.9882705741291365; alpha: 0.011729425870863452; period 84.75468154320117 (intervals)
   DEBUG: Exponential decay: 0.9882707323588271; alpha: 0.01172926764117288; period 84.75583166905389 (intervals)
   DEBUG: Exponential decay: 0.9882714847232316; alpha: 0.011728515276768436; period 84.76130081287236 (intervals)
   DEBUG: Exponential decay: 0.9882716704758148; alpha: 0.011728329524185233; period 84.76265120750082 (intervals)
   DEBUG: Exponential decay: 0.9882722940935008; alpha: 0.011727705906499186; period 

   DEBUG: Exponential decay: 0.9922940902005863; alpha: 0.007705909799413657; period 129.26988550995011 (intervals)
   DEBUG: Exponential decay: 0.99304243606623; alpha: 0.006957563933770006; period 143.22788542785605 (intervals)
   DEBUG: Exponential decay: 0.9925405267630246; alpha: 0.007459473236975378; period 133.5570994595063 (intervals)
   DEBUG: Exponential decay: 0.9922491170142791; alpha: 0.007750882985720908; period 128.51691020181116 (intervals)
   DEBUG: Exponential decay: 0.9924156103611456; alpha: 0.0075843896388544385; period 131.34913167311234 (intervals)
   DEBUG: Exponential decay: 0.9932532166123996; alpha: 0.006746783387600375; period 147.71821547281255 (intervals)
   DEBUG: Exponential decay: 0.991873862908576; alpha: 0.008126137091423957; period 122.55902096325596 (intervals)
   DEBUG: Exponential decay: 0.9917206870194987; alpha: 0.00827931298050133; period 120.2822758158384 (intervals)
   DEBUG: Exponential decay: 0.9923434797394763; alpha: 0.007656520260523658;

...

   DEBUG: Exponential decay: 0.9923207586692944; alpha: 0.007679241330705633; period 129.72055483677744 (intervals)
   DEBUG: Exponential decay: 0.9923602246106838; alpha: 0.00763977538931615; period 130.39326145391263 (intervals)
   DEBUG: Exponential decay: 0.9923423892528069; alpha: 0.0076576107471930666; period 130.08839468720842 (intervals)
   DEBUG: Exponential decay: 0.9923753809357783; alpha: 0.007624619064221738; period 130.6534554645215 (intervals)
   DEBUG: Exponential decay: 0.9924122623239794; alpha: 0.007587737676020612; period 131.29095358771482 (intervals)
   DEBUG: Exponential decay: 0.9924705285411204; alpha: 0.007529471458879611; period 132.3108172237707 (intervals)
   DEBUG: Exponential decay: 0.9924783003668469; alpha: 0.007521699633153078; period 132.44804582630692 (intervals)
   DEBUG: Exponential decay: 0.9925366547146526; alpha: 0.0074633452853474225; period 133.48754882287525 (intervals)
   DEBUG: Exponential decay: 0.9925097116914877; alpha: 0.007490288308512

   DEBUG: Exponential decay: 0.9947340275458656; alpha: 0.005265972454134404; period 189.39800872161308 (intervals)
   DEBUG: Exponential decay: 0.9948249229223967; alpha: 0.005175077077603341; period 192.73340453202704 (intervals)
   DEBUG: Exponential decay: 0.9948449285949822; alpha: 0.005155071405017808; period 193.48330326450593 (intervals)
   DEBUG: Exponential decay: 0.994877539382328; alpha: 0.0051224606176719645; period 194.7182520052635 (intervals)
   DEBUG: Exponential decay: 0.9947962269414675; alpha: 0.005203773058532479; period 191.66782251196972 (intervals)
   DEBUG: Exponential decay: 0.9949515555004738; alpha: 0.005048444499526239; period 197.5803930430677 (intervals)
   DEBUG: Exponential decay: 0.9948055181787062; alpha: 0.005194481821293784; period 192.0115497541891 (intervals)
   DEBUG: Exponential decay: 0.9947524893564222; alpha: 0.005247510643577757; period 190.06611166396047 (intervals)
   DEBUG: Exponential decay: 0.9948433262037445; alpha: 0.00515667379625550

   DEBUG: Exponential decay: 0.9964323096938513; alpha: 0.0035676903061486742; period 279.7930892513273 (intervals)
   DEBUG: Exponential decay: 0.9965982040045005; alpha: 0.0034017959954995014; period 293.4620821847732 (intervals)
   DEBUG: Exponential decay: 0.9963698655870318; alpha: 0.0036301344129682223; period 274.97159032830103 (intervals)
   DEBUG: Exponential decay: 0.9964227890423057; alpha: 0.0035772109576942546; period 279.0470950818833 (intervals)
   DEBUG: Exponential decay: 0.9964265510602867; alpha: 0.0035734489397133418; period 279.34139436073184 (intervals)
   DEBUG: Exponential decay: 0.9964919322573983; alpha: 0.003508067742601706; period 284.5569162357713 (intervals)
   DEBUG: Exponential decay: 0.9966636218450535; alpha: 0.0033363781549464733; period 299.22593764541347 (intervals)
   DEBUG: Exponential decay: 0.9968651666334784; alpha: 0.00313483336652165; period 318.49596018774486 (intervals)
   DEBUG: Exponential decay: 0.996757861058353; alpha: 0.00324213894164

   DEBUG: Exponential decay: 0.9989437099956056; alpha: 0.0010562900043944445; period 946.2096183922757 (intervals)
   DEBUG: Exponential decay: 0.9988726138184738; alpha: 0.001127386181526191; period 886.5074074084231 (intervals)
   DEBUG: Exponential decay: 0.9986953825230692; alpha: 0.0013046174769307806; period 766.0080958630147 (intervals)
   DEBUG: Exponential decay: 0.9988547176420909; alpha: 0.0011452823579091165; period 872.6470311452573 (intervals)
   DEBUG: Exponential decay: 0.9989290346100349; alpha: 0.001070965389965095; period 933.2369010597461 (intervals)
   DEBUG: Exponential decay: 0.9990296731778139; alpha: 0.0009703268221861183; period 1030.0805205382067 (intervals)
   DEBUG: Exponential decay: 0.9989076687905492; alpha: 0.0010923312094508475; period 914.9731567324808 (intervals)
   DEBUG: Exponential decay: 0.9988681912034979; alpha: 0.0011318087965020984; period 883.0413687195382 (intervals)
   DEBUG: Exponential decay: 0.9990338249891033; alpha: 0.000966175010896

DecayModel(start=1e-10, zero=4.911559813157194e-07, log10_scale=-6.737406500340045, log10_period=3.1559862902512146, input='HR Impulse / 10s', output='Fitness')


In [10]:
results = evaluate(data, result)
output_notebook()
f = figure(plot_width=500, plot_height=450, x_axis_type='datetime')
pt = Dark2[3]
# f.line(x=ff_data.index, y=ff_data[FITNESS], color=pt[0], legend=FITNESS)
f.line(x=results.index, y=results[FITNESS], color=pt[1], legend=f'new {FITNESS}')
f.circle(x=st_data.index, y=1.0/st_data[SEGMENT_TIME], fill_color=pt[2], size=10, legend='Speed')
f.legend.location = 'top_left'
show(f)

   DEBUG: Exponential decay: 0.9993019892767755; alpha: 0.000698010723224507; period 1432.1426888754897 (intervals)


Loading BokehJS ...

In [ ]:
x = (data[FITNESS] - data[SPEED]).dropna()
print(x)
x2 = x * x
print(sum(x2))

In [ ]:
print(hr10_data)